### 轉存 NNLM 的權重
+ 將 NNLM 的權重視為一般的 Numpy Array 做轉存

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

emb_layer = hub.KerasLayer('./NNLM-ZH')
np.save('emb.npy', emb_layer.get_weights()[0])
del emb_layer

### 讀取權重後放入 Keras Embedding Layer
+ 使用 `tf.keras.initializers.Constant` 將 Numpy Array 轉成 Layer Weights 即可使用

In [2]:
inn = tf.keras.layers.Input(shape=(10,))

# Load embedding weights
emb_weights = np.load('emb.npy')
emb_const = tf.keras.initializers.Constant(emb_weights)
emb = tf.keras.layers.Embedding(
    emb_weights.shape[0],
    emb_weights.shape[1],
    embeddings_constraint=emb_const,
    trainable=False # Optimal
)(inn)

hid = tf.keras.layers.LSTM(128)(emb)
out = tf.keras.layers.Dense(1)(hid)

model = tf.keras.Model(inn, out)
model.compile('adam', 'binary_crossentropy', ['acc'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 10, 128)           124310656 
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 124,442,369
Trainable params: 131,713
Non-trainable params: 124,310,656
_________________________________________________________________


In [3]:
x = np.random.randint(0, emb_weights.shape[0] - 1, (10000, 10))
y = np.random.randint(0, 2, (10000,))

model.fit(x, y, epochs=2);

Train on 10000 samples
Epoch 1/2
10000/10000 [==============================] - 4s 383us/sample - loss: 0.7824 - acc: 0.4973
Epoch 2/2
10000/10000 [==============================] - 2s 166us/sample - loss: 0.6971 - acc: 0.5088
